In [1]:
from common_functions import ret_metabase, google_sheets, dwh_query, upload_dataframe_to_pg, upload_dataframe_to_snowflake, snowflake_query, get_secret
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd


import numpy as np
import gspread
import pytz


df = ret_metabase("Egypt", 15799)
active_df = df[df["SELLING_SCHEME_ACTIVITY"] == "ACTIVE"]
df_filtered = active_df[["SP","SERVICE_PROVIDER_ID", "BRAND","BRAND_ID", "SELLING_SCHEME_ID", "PURCHASE_ORDER"]]

def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token


import requests

AT = get_jwt()  # get token once (unless it expires quickly)
url = "https://emoney.maxab.info/emoney/api/v1/purchase-orders"

# Drop rows with NaN in required columns
df_filtered = df_filtered.dropna(
    subset=["BRAND_ID", "PURCHASE_ORDER", "SELLING_SCHEME_ID", "SERVICE_PROVIDER_ID"]
).reset_index(drop=True)

for idx, row in df_filtered.iterrows():
    payload = {
        "brandId": {
            "id": int(row["BRAND_ID"]),
            "status": {
                "id": 2,
                "key": "active",
                "displayName": "ACTIVE"
            },
            "names": {"en": row["BRAND"]}
        },
        "quantity": int(row["PURCHASE_ORDER"]),
        "sellingSchemaId": int(row["SELLING_SCHEME_ID"]),
        "serviceProviderWrapperId": {
            "id": int(row["SERVICE_PROVIDER_ID"]),
            "officialEnglishName": row["SP"]
        }
    }

    headers = {
        "authorization": f"Bearer {AT}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)
    print(f"Row {idx} → Status: {response.status_code}, Response: {response.text}")


In [7]:
fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
fintech_service_account_emailname= fintech_service_account["email_name"]
fintech_service_account_emailpass= fintech_service_account["email_password"]
payload = {
        "client_id":"admin-portal",
        "grant_type":"password",
        "username":fintech_service_account_emailname,
        "password":fintech_service_account_emailpass 
    }


In [2]:
df = ret_metabase("Egypt", 15799)
df

/home/ec2-user/service_account_key.json


,SP,SERVICE_PROVIDER_ID,SERVICE,BRAND,BRAND_ID,SELLING_SCHEME_NAME,SELLING_SCHEME_ID,SELLING_SCHEME_ACTIVITY,MAIN_SOURCE,CURRENT_STOCK,INPROGRESS_POS,AVG_DAILY_ROLLING_7_DAYS,TWO_DAYS_PO_COUNT,THREE_DAYS_PO_COUNT,FOUR_DAYS_PO_COUNT,PURCHASE_ORDER
0,Superpay,300,Vouchers,Etisalat,7,Etisalat_Voucher_16.5EGP,16732,ACTIVE,VWH,13434.0,1.0,51561,89688.0,141249.0,192810.0,89688.0
1,Superpay,300,Vouchers,Etisalat,7,Etisalat_Voucher_19.5EGP,16765,ACTIVE,VWH,9018.0,0.0,20171,31324.0,51495.0,71666.0,31324.0
2,Masary,7,Vouchers,Vodafone,6,Vodafone_Voucher_16.5EGP,15585,ACTIVE,VWH,10073.0,0.0,19514,28955.0,48469.0,67983.0,28955.0
3,Superpay,300,Bulk Printing,Etisalat,7,Bulk_Etisalat_16.5,16803,ACTIVE,VWH,8548.0,0.0,18087,27626.0,45713.0,63800.0,27626.0
4,Superpay,300,Vouchers,Etisalat,7,Etisalat_Voucher_26EGP,16798,ACTIVE,VWH,5986.0,0.0,15396,24806.0,40202.0,55598.0,24806.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Superpay,300,Vouchers,Etisalat,7,HekayaVoucher235,16852,ACTIVE,No-Vstock - API,NaN,NaN,3,NaN,NaN,NaN,NaN
86,Masary,7,Vouchers,Vodafone,6,Flexvoucher130,16091,ACTIVE,No-Vstock - API,NaN,NaN,3,NaN,NaN,NaN,NaN
87,Damen,399,Bulk Printing,WE,9,Bulk_WE_40EGP,22522,ACTIVE,API_Auto_Switched,0.0,0.0,3,6.0,9.0,12.0,6.0
88,Masary,7,Vouchers,WE,9,Voucher_WE_30EGP,20525,INACTIVE,API_Auto_Switched,0.0,0.0,2,4.0,6.0,8.0,4.0


In [3]:
active_df = df[df["SELLING_SCHEME_ACTIVITY"] == "ACTIVE"]

In [4]:
df_filtered = active_df[["SP","SERVICE_PROVIDER_ID", "BRAND","BRAND_ID", "SELLING_SCHEME_ID", "PURCHASE_ORDER"]]

In [31]:
df_filtered = df_filtered.head(2)
df_filtered

,SP,SERVICE_PROVIDER_ID,BRAND,BRAND_ID,SELLING_SCHEME_ID,PURCHASE_ORDER
0,Superpay,300,Etisalat,7,16732,101532.0
1,Superpay,300,Etisalat,7,16765,33211.0


In [5]:
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token

In [23]:
for i in range(3):
    AT = get_jwt()
    url = "https://emoney.maxab.info/emoney/api/v1/purchase-orders"

    payload = {
        "brandId": {
            "id": df_filtered["BRAND_ID"][i],
            "status": {
                "id": 2,
                "key": "active",
                "displayName": "ACTIVE"
            },
            "names": {"en": df_filtered["BRAND"][i]}
        },
        "quantity": df_filtered["PURCHASE_ORDER"][i],
        "sellingSchemaId": df_filtered["SELLING_SCHEME_ID"][i],
        "serviceProviderWrapperId": {
            "id": df_filtered["SERVICE_PROVIDER_ID"][i],
            "officialEnglishName": df_filtered["SP"][i]
        }
    }
    headers = {
        "authorization": f"Bearer {AT}",
        "Content-Type": "application/json"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4cnE5b1ZqaXRFdjdNeExyTWpzS3NkdUl3eWdxZnZlVkM4T0Vfb3dvY3N3In0.eyJleHAiOjE3NTYzODg3NDAsImlhdCI6MTc1NjM4ODQ0MCwianRpIjoiOGRhOGE3Y2EtNTAwYi00MjcwLThmMjAtNjNkNWE4NTA4ZmFjIiwiaXNzIjoiaHR0cHM6Ly9zc28ubWF4YWIuaW5mby9hdXRoL3JlYWxtcy9tYXhhYiIsImF1ZCI6WyJ3b3JrZm9yY2UiLCJlLW1vbmV5Iiwib3JkZXItbWFuYWdlbWVudCIsImNyZWRpdCIsImFjY291bnQiXSwic3ViIjoiZGM1ZGY3MDEtZjg3ZC00MDg1LTg3ZmMtNWMyNGVmM2E0YjhkIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWRtaW4tcG9ydGFsIiwic2Vzc2lvbl9zdGF0ZSI6ImVjNjc5MzNiLTdlOGItNDM4Mi1iYmQwLWI3NjYwNDY3MmVmZCIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwczovL2FkbWluLm1heGFiLmluZm8iXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImRlZmF1bHQtcm9sZXMtbWF4YWIiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsid29ya2ZvcmNlIjp7InJvbGVzIjpbIm9yZGVyLW1hbmFnbWVudC1hcGlzLWludGVncmF0aW9uIiwid29ya2ZvcmNlLWludGVncmF0aW9uIl19LCJlLW1vbmV5Ijp7InJvbGVzIjpbImUtbW9uZXkiXX0sIm9yZGVyLW1hbmFnZW1lbnQiOnsicm9sZXMiOlsidW1hX3Byb3RlY3R

In [ ]:
AT = get_jwt()
url = "https://emoney.maxab.info/emoney/api/v1/purchase-orders"

payload = {
    "brandId": {
        "id": 6,
        "status": {
            "id": 2,
            "key": "active",
            "displayName": "ACTIVE"
        },
        "names": {"en": "Vodafone"}
    },
    "quantity": 1,
    "sellingSchemaId": 15585,
    "serviceProviderWrapperId": {
        "id": 7,
        "officialEnglishName": "Masary"
    }
}
headers = {
    "authorization": f"Bearer {AT}",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

In [7]:
df_filtered.head(20)

,SP,SERVICE_PROVIDER_ID,BRAND,BRAND_ID,SELLING_SCHEME_ID,PURCHASE_ORDER
0,Superpay,300,Etisalat,7,16732,94575.0
1,Superpay,300,Etisalat,7,16765,28859.0
2,Masary,7,Vodafone,6,15585,28484.0
3,Superpay,300,Etisalat,7,16803,27751.0
4,Superpay,300,Etisalat,7,16798,21553.0
5,Masary,7,Vodafone,6,15548,14845.0
6,Masary,7,Vodafone,6,15550,11985.0
7,Masary,7,Vodafone,6,15547,11898.0
8,Orange,5,Orange,8,14889,11879.0
9,Orange,5,Orange,8,14890,7278.0


In [14]:

# df_filtered = df_filtered.dropna(
#     subset=["BRAND_ID", "PURCHASE_ORDER", "SELLING_SCHEME_ID", "SERVICE_PROVIDER_ID"]
# )

# df_filtered = df_filtered.reset_index(drop=True)

df_filtered

,SP,SERVICE_PROVIDER_ID,BRAND,BRAND_ID,SELLING_SCHEME_ID,PURCHASE_ORDER
0,Superpay,300,Etisalat,7,16732,94575.0
1,Superpay,300,Etisalat,7,16765,28859.0
2,Masary,7,Vodafone,6,15585,28484.0
3,Superpay,300,Etisalat,7,16803,27751.0
4,Superpay,300,Etisalat,7,16798,21553.0
...,...,...,...,...,...,...
62,Superpay,300,Etisalat,7,16807,6.0
63,Damen,399,WE,9,22525,1.0
64,Damen,399,WE,9,22520,1.0
65,Damen,399,WE,9,22522,6.0


In [6]:
import requests

AT = get_jwt()  # get token once (unless it expires quickly)
url = "https://emoney.maxab.info/emoney/api/v1/purchase-orders"

# Drop rows with NaN in required columns
df_filtered = df_filtered.dropna(
    subset=["BRAND_ID", "PURCHASE_ORDER", "SELLING_SCHEME_ID", "SERVICE_PROVIDER_ID"]
).reset_index(drop=True)

for idx, row in df_filtered.iterrows():
    payload = {
        "brandId": {
            "id": int(row["BRAND_ID"]),
            "status": {
                "id": 2,
                "key": "active",
                "displayName": "ACTIVE"
            },
            "names": {"en": row["BRAND"]}
        },
        "quantity": int(row["PURCHASE_ORDER"]),
        "sellingSchemaId": int(row["SELLING_SCHEME_ID"]),
        "serviceProviderWrapperId": {
            "id": int(row["SERVICE_PROVIDER_ID"]),
            "officialEnglishName": row["SP"]
        }
    }

    headers = {
        "authorization": f"Bearer {AT}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)
    print(f"Row {idx} → Status: {response.status_code}, Response: {response.text}")


Row 0 → Status: 200, Response: 
Row 1 → Status: 200, Response: 
Row 2 → Status: 200, Response: 
Row 3 → Status: 200, Response: 
Row 4 → Status: 200, Response: 
Row 5 → Status: 200, Response: 
Row 6 → Status: 200, Response: 
Row 7 → Status: 200, Response: 
Row 8 → Status: 200, Response: 
Row 9 → Status: 200, Response: 
Row 10 → Status: 200, Response: 
Row 11 → Status: 200, Response: 
Row 12 → Status: 200, Response: 
Row 13 → Status: 200, Response: 
Row 14 → Status: 200, Response: 
Row 15 → Status: 200, Response: 
Row 16 → Status: 200, Response: 
Row 17 → Status: 200, Response: 
Row 18 → Status: 200, Response: 
Row 19 → Status: 200, Response: 
Row 20 → Status: 200, Response: 
Row 21 → Status: 200, Response: 
Row 22 → Status: 200, Response: 
Row 23 → Status: 200, Response: 
Row 24 → Status: 200, Response: 
Row 25 → Status: 200, Response: 
Row 26 → Status: 200, Response: 
Row 27 → Status: 200, Response: 
Row 28 → Status: 200, Response: 
Row 29 → Status: 200, Response: 
Row 30 → Status: 200